In [6]:
import tensorflow as tf
from pickle import FALSE
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_shape = (50,50)
batch = 32

TRAINING_DIR = "dataset/train"
training_datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=True,
      samplewise_std_normalization=True,
      zca_whitening=False,
      rotation_range = 30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip = False,
      vertical_flip=False
)

VALIDATION_DIR = "dataset/val"
validation_datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=True,
      samplewise_std_normalization=True,
      zca_whitening=False,
      rotation_range = 30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=False,
      vertical_flip=False
)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=image_shape,
  interpolation='nearest',
  color_mode='grayscale',
	class_mode='categorical',
  batch_size=batch
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=image_shape,
  interpolation='nearest',
  color_mode='grayscale',
	class_mode='categorical',
  batch_size=batch
)

Found 28800 images belonging to 17 classes.
Found 7200 images belonging to 17 classes.


In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = tf.keras.models.Sequential([
  # ALEX-NET

  tf.keras.layers.Conv2D(96, (11,11), strides=4, padding='same', activation='relu', input_shape=(image_shape[0],image_shape[1],1)),
  tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='valid'),

  tf.keras.layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='valid'),

  tf.keras.layers.Conv2D(384, (3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.Conv2D(384, (3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.Conv2D(256, (3,3), strides=1, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='valid'),

  # Top
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dropout(0.5),  # dropout after dense
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dropout(0.5),  # dropout after dense
  tf.keras.layers.Dense(17, activation='softmax')
])
model.summary()

c:\Python\python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 57, 57, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 28, 28, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 256)    │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 17)             │        69,649 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,934,545 (221.00 MB)

 Trainable params: 57,934,545 (221.00 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from keras.optimizers import Adam
# Create optimizer with initial learning rate
optimizer = Adam(learning_rate=6.5610e-11)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

saved_model = tf.keras.models.load_model('model-alexnet-16.keras')
saved_params = saved_model.get_weights()
model.set_weights(saved_params)

learning_rate_callback = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1, factor=0.3)
history = model.fit(train_generator, epochs=16, validation_data = validation_generator, callbacks = [learning_rate_callback])

Epoch 1/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 516s 2s/step - accuracy: 0.9663 - loss: 0.1124 - val_accuracy: 0.9701 - val_loss: 0.0937 - learning_rate: 8.1000e-09
Epoch 2/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - accuracy: 0.9666 - loss: 0.1051 - val_accuracy: 0.9718 - val_loss: 0.0914 - learning_rate: 8.1000e-09
Epoch 3/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.9671 - loss: 0.1091 - val_accuracy: 0.9719 - val_loss: 0.0929 - learning_rate: 8.1000e-09
Epoch 4/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.9668 - loss: 0.1088 - val_accuracy: 0.9742 - val_loss: 0.0908 - learning_rate: 8.1000e-09
Epoch 5/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - accuracy: 0.9679 - loss: 0.1103 - val_accuracy: 0.9744 - val_loss: 0.0835 - learning_rate: 8.1000e-09
Epoch 6/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - accuracy: 0.9677 - loss: 0.1168 - val_accuracy: 0.9760 - val_loss: 0.0812 - learning_rate: 8.1000e-09
Epoch 7/16
225/225 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - acc

In [9]:
model.save("model-alexnet.keras")

In [1]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: name 'history' is not defined

In [1]:
import cv2
import numpy as np

# Create a 500x500 white image
white_image = np.ones((500, 500, 3), np.uint8) * 255
cv2.rectangle(white_image,(10,10),(250,100),(255,0,0),2)
# Save the image
cv2.imshow('white_image.jpg', white_image)

# Wait for a key press to close the window
cv2.waitKey(0)

# Close all windows
cv2.destroyAllWindows()